In [1]:
!pip install dgl-cu101 dglgo -f https://data.dgl.ai/wheels/repo.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.dgl.ai/wheels/repo.html


You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time

from scipy.sparse import coo_matrix
from dgl.nn import GraphConv

### Optional (GPU Training / Inference)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define our Graph Convolutional Network

In [4]:
class GCN(nn.Module):
    def __init__(self, l1_in_feats, l1_out_feats, l2_in_feats, l2_out_feats):
        super(GCN, self).__init__()
        self.layer1 = GraphConv(l1_in_feats, l1_out_feats)
        self.layer2 = GraphConv(l2_in_feats, l2_out_feats)
    
    def forward(self, g, in_feats):
        h = F.relu(self.layer1(g, in_feats))
        h = self.layer2(g, h)
        return h
        

In [5]:
def load_dataset(d,device):
    g = d[0].to(device)
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask

In [6]:
def evaluate(model, g, features, labels, test_mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[test_mask]
        labels = labels[test_mask]
        _,indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [7]:
def train(model, g, feats, labels, train_mask, test_mask, num_epoch):
    opt = torch.optim.Adam(model.parameters(), lr=1e-2)
    best_test_acc = 0
    
    # Add edges between each node and itself to preserve old node representations
    g.add_edges(g.nodes(), g.nodes())
    duration = []
    for epoch in range(num_epoch):
        t0 = time.time()

        model.train()
        logits = model(g, feats)
        logp = F.log_softmax(logits, 1)
        loss = F.nll_loss(logp[train_mask], labels[train_mask])
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        duration.append(time.time() - t0)
        test_acc = evaluate(model, g, feats, labels, test_mask)
        if test_acc > best_test_acc:
            best_test_acc = test_acc
    
        print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f} | (Best {:.4f})" 
              .format(epoch, loss.item(), test_acc, np.mean(duration), best_test_acc))
    return model

In [8]:
def save_weight(filename, coo_data):
    with open(filename, 'w') as f:
        hidden_size = len(coo_data.data)
        
        print(str(coo_data.shape[0])+" "+
              str(coo_data.shape[1]), file=f)
        
        for i in range(hidden_size):
            print(str(coo_data.row[i]) + " " +
                  str(coo_data.col[i]) + " " +
                  str(coo_data.data[i]), file=f)

In [9]:
activation = {}
def get_activation(name):
    def hook(model, inp, out):
        activation[name] = out.detach()
    return hook

# Cora Dataset

In [10]:
dataset = dgl.data.CoraGraphDataset() # import dataset

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [11]:
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
model = GCN(1433,16,16,7).to(device)

# Register hook to get the hidden features
model.layer1.register_forward_hook(get_activation('layer1'))
model.layer1.register_forward_hook(get_activation('layer2'))

In [12]:
train(model, g, feats, labels, train_mask, test_mask, 1000)

Epoch 00000 | Loss 1.9452 | Test Acc 0.1570 | Time(s) 0.0125 | (Best 0.1570)
Epoch 00001 | Loss 1.9384 | Test Acc 0.3830 | Time(s) 0.0088 | (Best 0.3830)
Epoch 00002 | Loss 1.9291 | Test Acc 0.5930 | Time(s) 0.0075 | (Best 0.5930)
Epoch 00003 | Loss 1.9160 | Test Acc 0.7120 | Time(s) 0.0068 | (Best 0.7120)
Epoch 00004 | Loss 1.9029 | Test Acc 0.7060 | Time(s) 0.0064 | (Best 0.7120)
Epoch 00005 | Loss 1.8892 | Test Acc 0.7060 | Time(s) 0.0061 | (Best 0.7120)
Epoch 00006 | Loss 1.8743 | Test Acc 0.6750 | Time(s) 0.0059 | (Best 0.7120)
Epoch 00007 | Loss 1.8586 | Test Acc 0.6490 | Time(s) 0.0058 | (Best 0.7120)
Epoch 00008 | Loss 1.8420 | Test Acc 0.6490 | Time(s) 0.0057 | (Best 0.7120)
Epoch 00009 | Loss 1.8244 | Test Acc 0.6590 | Time(s) 0.0056 | (Best 0.7120)
Epoch 00010 | Loss 1.8057 | Test Acc 0.6830 | Time(s) 0.0056 | (Best 0.7120)
Epoch 00011 | Loss 1.7860 | Test Acc 0.7040 | Time(s) 0.0055 | (Best 0.7120)
Epoch 00012 | Loss 1.7652 | Test Acc 0.7280 | Time(s) 0.0054 | (Best 0.7280)

Epoch 00130 | Loss 0.0483 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00131 | Loss 0.0474 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00132 | Loss 0.0465 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00133 | Loss 0.0456 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00134 | Loss 0.0448 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00135 | Loss 0.0439 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00136 | Loss 0.0431 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00137 | Loss 0.0423 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00138 | Loss 0.0416 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00139 | Loss 0.0408 | Test Acc 0.7790 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00140 | Loss 0.0401 | Test Acc 0.7790 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00141 | Loss 0.0394 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00142 | Loss 0.0387 | Test Acc 0.7780 | Time(s) 0.0049 | (Best 0.7830)

Epoch 00260 | Loss 0.0100 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00261 | Loss 0.0099 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00262 | Loss 0.0098 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00263 | Loss 0.0098 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00264 | Loss 0.0097 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00265 | Loss 0.0096 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00266 | Loss 0.0095 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00267 | Loss 0.0095 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00268 | Loss 0.0094 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00269 | Loss 0.0093 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00270 | Loss 0.0093 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00271 | Loss 0.0092 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00272 | Loss 0.0091 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)

Epoch 00390 | Loss 0.0044 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00391 | Loss 0.0044 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00392 | Loss 0.0044 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00393 | Loss 0.0043 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00394 | Loss 0.0043 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00395 | Loss 0.0043 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00396 | Loss 0.0043 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00397 | Loss 0.0043 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00398 | Loss 0.0042 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00399 | Loss 0.0042 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00400 | Loss 0.0042 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00401 | Loss 0.0042 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00402 | Loss 0.0042 | Test Acc 0.7710 | Time(s) 0.0049 | (Best 0.7830)

Epoch 00520 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00521 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00522 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00523 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00524 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00525 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00526 | Loss 0.0025 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00527 | Loss 0.0024 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00528 | Loss 0.0024 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00529 | Loss 0.0024 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00530 | Loss 0.0024 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00531 | Loss 0.0024 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00532 | Loss 0.0024 | Test Acc 0.7740 | Time(s) 0.0049 | (Best 0.7830)

Epoch 00649 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00650 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00651 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00652 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00653 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00654 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00655 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00656 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00657 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00658 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00659 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00660 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00661 | Loss 0.0016 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)

Epoch 00779 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00780 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00781 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00782 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00783 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00784 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00785 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00786 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00787 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00788 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00789 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00790 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)
Epoch 00791 | Loss 0.0011 | Test Acc 0.7750 | Time(s) 0.0049 | (Best 0.7830)

Epoch 00909 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00910 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00911 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00912 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00913 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00914 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00915 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00916 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00917 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00918 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00919 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00920 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)
Epoch 00921 | Loss 0.0008 | Test Acc 0.7740 | Time(s) 0.0048 | (Best 0.7830)

GCN(
  (layer1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (layer2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

## Save Hidden Features

In [13]:
l1_feats = activation['layer1'].cpu()
l2_feats = activation['layer2'].cpu()

save_weight('data/features/Cora_feat_L1.txt',coo_matrix(l1_feats))
save_weight('data/features/Cora_feat_L2.txt',coo_matrix(l2_feats))

activation.clear()
### Save Layer Weight (State Dict) ###

torch.save(model.state_dict(),'data/weight/cora.pt')
#statedict = model.state_dict()
#L1 = statedict['layer1.weight'].to('cpu') + statedict['layer1.bias'].to('cpu')
#L2 = statedict['layer2.weight'].to('cpu') + statedict['layer2.bias'].to('cpu')



# Citeseer Dataset

In [14]:
dataset = dgl.data.CiteseerGraphDataset() # import dataset

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [15]:
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
model = GCN(3703,16,16,6).to(device)

# Register hook to get the hidden features
model.layer1.register_forward_hook(get_activation('layer1'))
model.layer1.register_forward_hook(get_activation('layer2'))

In [16]:
train(model, g, feats, labels, train_mask, test_mask, 1000)

Epoch 00000 | Loss 1.7917 | Test Acc 0.2390 | Time(s) 0.0070 | (Best 0.2390)
Epoch 00001 | Loss 1.7853 | Test Acc 0.3480 | Time(s) 0.0059 | (Best 0.3480)
Epoch 00002 | Loss 1.7760 | Test Acc 0.4900 | Time(s) 0.0055 | (Best 0.4900)
Epoch 00003 | Loss 1.7635 | Test Acc 0.5070 | Time(s) 0.0053 | (Best 0.5070)
Epoch 00004 | Loss 1.7494 | Test Acc 0.5150 | Time(s) 0.0052 | (Best 0.5150)
Epoch 00005 | Loss 1.7352 | Test Acc 0.5170 | Time(s) 0.0051 | (Best 0.5170)
Epoch 00006 | Loss 1.7204 | Test Acc 0.5470 | Time(s) 0.0050 | (Best 0.5470)
Epoch 00007 | Loss 1.7044 | Test Acc 0.5670 | Time(s) 0.0050 | (Best 0.5670)
Epoch 00008 | Loss 1.6873 | Test Acc 0.5880 | Time(s) 0.0050 | (Best 0.5880)
Epoch 00009 | Loss 1.6687 | Test Acc 0.6050 | Time(s) 0.0049 | (Best 0.6050)
Epoch 00010 | Loss 1.6488 | Test Acc 0.6060 | Time(s) 0.0049 | (Best 0.6060)
Epoch 00011 | Loss 1.6274 | Test Acc 0.6050 | Time(s) 0.0049 | (Best 0.6060)
Epoch 00012 | Loss 1.6051 | Test Acc 0.6090 | Time(s) 0.0049 | (Best 0.6090)

Epoch 00126 | Loss 0.0369 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00127 | Loss 0.0362 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00128 | Loss 0.0355 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00129 | Loss 0.0348 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00130 | Loss 0.0342 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00131 | Loss 0.0335 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00132 | Loss 0.0329 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00133 | Loss 0.0323 | Test Acc 0.6360 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00134 | Loss 0.0318 | Test Acc 0.6370 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00135 | Loss 0.0312 | Test Acc 0.6370 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00136 | Loss 0.0307 | Test Acc 0.6350 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00137 | Loss 0.0301 | Test Acc 0.6350 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00138 | Loss 0.0296 | Test Acc 0.6340 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00234 | Loss 0.0094 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00235 | Loss 0.0093 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00236 | Loss 0.0092 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00237 | Loss 0.0092 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00238 | Loss 0.0091 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00239 | Loss 0.0090 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00240 | Loss 0.0089 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00241 | Loss 0.0088 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00242 | Loss 0.0088 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00243 | Loss 0.0087 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00244 | Loss 0.0086 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00245 | Loss 0.0086 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00246 | Loss 0.0085 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00360 | Loss 0.0040 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00361 | Loss 0.0040 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00362 | Loss 0.0040 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00363 | Loss 0.0039 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00364 | Loss 0.0039 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00365 | Loss 0.0039 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00366 | Loss 0.0039 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00367 | Loss 0.0038 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00368 | Loss 0.0038 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00369 | Loss 0.0038 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00370 | Loss 0.0038 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00371 | Loss 0.0038 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00372 | Loss 0.0037 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00468 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00469 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00470 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00471 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00472 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00473 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00474 | Loss 0.0024 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00475 | Loss 0.0023 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00476 | Loss 0.0023 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00477 | Loss 0.0023 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00478 | Loss 0.0023 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00479 | Loss 0.0023 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00480 | Loss 0.0023 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00594 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00595 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00596 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00597 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00598 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00599 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00600 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00601 | Loss 0.0015 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00602 | Loss 0.0015 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00603 | Loss 0.0015 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00604 | Loss 0.0015 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00605 | Loss 0.0015 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00606 | Loss 0.0015 | Test Acc 0.6290 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00702 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00703 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00704 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00705 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00706 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00707 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00708 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00709 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00710 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00711 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00712 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00713 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00714 | Loss 0.0011 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00828 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00829 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00830 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00831 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00832 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00833 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00834 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00835 | Loss 0.0008 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00836 | Loss 0.0008 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00837 | Loss 0.0008 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00838 | Loss 0.0008 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00839 | Loss 0.0008 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00840 | Loss 0.0008 | Test Acc 0.6300 | Time(s) 0.0048 | (Best 0.6650)

Epoch 00936 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00937 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00938 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00939 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00940 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00941 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00942 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00943 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00944 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00945 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00946 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00947 | Loss 0.0006 | Test Acc 0.6320 | Time(s) 0.0048 | (Best 0.6650)
Epoch 00948 | Loss 0.0006 | Test Acc 0.6310 | Time(s) 0.0048 | (Best 0.6650)

GCN(
  (layer1): GraphConv(in=3703, out=16, normalization=both, activation=None)
  (layer2): GraphConv(in=16, out=6, normalization=both, activation=None)
)

## Save Hidden Features

In [17]:
l1_feats = activation['layer1'].cpu()
l2_feats = activation['layer2'].cpu()

save_weight('data/features/Citeseer_feat_L1.txt',coo_matrix(l1_feats))
save_weight('data/features/Citeseer_feat_L2.txt',coo_matrix(l2_feats))

activation.clear()

torch.save(model.state_dict(),'data/weight/citeseer.pt')

### Save Layer Weight (State Dict) ###

#statedict = model.state_dict()
#L1 = statedict['layer1.weight'].to('cpu') + statedict['layer1.bias'].to('cpu')
#L2 = statedict['layer2.weight'].to('cpu') + statedict['layer2.bias'].to('cpu')



# Pubmed Dataset

In [18]:
dataset = dgl.data.PubmedGraphDataset() # import dataset

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [19]:
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
model = GCN(500,16,16,3).to(device)

# Register hook to get the hidden features
model.layer1.register_forward_hook(get_activation('layer1'))
model.layer1.register_forward_hook(get_activation('layer2'))

In [20]:
train(model, g, feats, labels, train_mask, test_mask, 1000)

Epoch 00000 | Loss 1.0990 | Test Acc 0.4020 | Time(s) 0.0071 | (Best 0.4020)
Epoch 00001 | Loss 1.0913 | Test Acc 0.4380 | Time(s) 0.0060 | (Best 0.4380)
Epoch 00002 | Loss 1.0845 | Test Acc 0.5870 | Time(s) 0.0056 | (Best 0.5870)
Epoch 00003 | Loss 1.0763 | Test Acc 0.6510 | Time(s) 0.0054 | (Best 0.6510)
Epoch 00004 | Loss 1.0667 | Test Acc 0.6700 | Time(s) 0.0053 | (Best 0.6700)
Epoch 00005 | Loss 1.0567 | Test Acc 0.6850 | Time(s) 0.0052 | (Best 0.6850)
Epoch 00006 | Loss 1.0465 | Test Acc 0.6870 | Time(s) 0.0052 | (Best 0.6870)
Epoch 00007 | Loss 1.0356 | Test Acc 0.6890 | Time(s) 0.0051 | (Best 0.6890)
Epoch 00008 | Loss 1.0239 | Test Acc 0.6940 | Time(s) 0.0051 | (Best 0.6940)
Epoch 00009 | Loss 1.0112 | Test Acc 0.6980 | Time(s) 0.0051 | (Best 0.6980)
Epoch 00010 | Loss 0.9976 | Test Acc 0.7050 | Time(s) 0.0050 | (Best 0.7050)
Epoch 00011 | Loss 0.9830 | Test Acc 0.7130 | Time(s) 0.0050 | (Best 0.7130)
Epoch 00012 | Loss 0.9676 | Test Acc 0.7230 | Time(s) 0.0050 | (Best 0.7230)

Epoch 00121 | Loss 0.0269 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00122 | Loss 0.0264 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00123 | Loss 0.0258 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00124 | Loss 0.0253 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00125 | Loss 0.0248 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00126 | Loss 0.0243 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00127 | Loss 0.0238 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00128 | Loss 0.0234 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00129 | Loss 0.0229 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00130 | Loss 0.0225 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00131 | Loss 0.0221 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00132 | Loss 0.0217 | Test Acc 0.7640 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00133 | Loss 0.0213 | Test Acc 0.7640 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00234 | Loss 0.0060 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00235 | Loss 0.0060 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00236 | Loss 0.0059 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00237 | Loss 0.0059 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00238 | Loss 0.0058 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00239 | Loss 0.0058 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00240 | Loss 0.0057 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00241 | Loss 0.0057 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00242 | Loss 0.0056 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00243 | Loss 0.0056 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00244 | Loss 0.0055 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00245 | Loss 0.0055 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00246 | Loss 0.0054 | Test Acc 0.7650 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00355 | Loss 0.0026 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00356 | Loss 0.0026 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00357 | Loss 0.0025 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00358 | Loss 0.0025 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00359 | Loss 0.0025 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00360 | Loss 0.0025 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00361 | Loss 0.0025 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00362 | Loss 0.0025 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00363 | Loss 0.0025 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00364 | Loss 0.0024 | Test Acc 0.7680 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00365 | Loss 0.0024 | Test Acc 0.7680 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00366 | Loss 0.0024 | Test Acc 0.7680 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00367 | Loss 0.0024 | Test Acc 0.7680 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00468 | Loss 0.0015 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00469 | Loss 0.0015 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00470 | Loss 0.0015 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00471 | Loss 0.0015 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00472 | Loss 0.0015 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00473 | Loss 0.0015 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00474 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00475 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00476 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00477 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00478 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00479 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00480 | Loss 0.0014 | Test Acc 0.7660 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00587 | Loss 0.0010 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00588 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00589 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00590 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00591 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00592 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00593 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00594 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00595 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00596 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00597 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00598 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00599 | Loss 0.0009 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00697 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00698 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00699 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00700 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00701 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00702 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00703 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00704 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00705 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00706 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00707 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00708 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00709 | Loss 0.0007 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00818 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00819 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00820 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00821 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00822 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00823 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00824 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00825 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00826 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00827 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00828 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00829 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00830 | Loss 0.0005 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)

Epoch 00931 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00932 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00933 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00934 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00935 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00936 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00937 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00938 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00939 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00940 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00941 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00942 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)
Epoch 00943 | Loss 0.0004 | Test Acc 0.7670 | Time(s) 0.0049 | (Best 0.7720)

GCN(
  (layer1): GraphConv(in=500, out=16, normalization=both, activation=None)
  (layer2): GraphConv(in=16, out=3, normalization=both, activation=None)
)

## Save Hidden Features

In [21]:
l1_feats = activation['layer1'].cpu()
l2_feats = activation['layer2'].cpu()

save_weight('data/features/Pubmed_feat_L1.txt',coo_matrix(l1_feats))
save_weight('data/features/Pubmed_feat_L2.txt',coo_matrix(l2_feats))

activation.clear()
torch.save(model.state_dict(),'data/weight/pubmed.pt')

### Save Layer Weight (State Dict) ###

#statedict = model.state_dict()
#L1 = statedict['layer1.weight'].to('cpu') + statedict['layer1.bias'].to('cpu')
#L2 = statedict['layer2.weight'].to('cpu') + statedict['layer2.bias'].to('cpu')



# Reddit Dataset

In [22]:
dataset = dgl.data.RedditDataset() # import dataset

In [23]:
g, feats, labels, train_mask, test_mask = load_dataset(dataset, device)
model = GCN(602,16,16,41).to(device)

# Register hook to get the hidden features
model.layer1.register_forward_hook(get_activation('layer1'))
model.layer1.register_forward_hook(get_activation('layer2'))

In [24]:
train(model, g, feats, labels, train_mask, test_mask, 1000)

Epoch 00000 | Loss 3.6900 | Test Acc 0.1586 | Time(s) 0.3802 | (Best 0.1586)
Epoch 00001 | Loss 3.4608 | Test Acc 0.1875 | Time(s) 0.2315 | (Best 0.1875)
Epoch 00002 | Loss 3.2845 | Test Acc 0.2035 | Time(s) 0.1822 | (Best 0.2035)
Epoch 00003 | Loss 3.1766 | Test Acc 0.2234 | Time(s) 0.1573 | (Best 0.2234)
Epoch 00004 | Loss 3.0676 | Test Acc 0.2524 | Time(s) 0.1424 | (Best 0.2524)
Epoch 00005 | Loss 2.9569 | Test Acc 0.3048 | Time(s) 0.1325 | (Best 0.3048)
Epoch 00006 | Loss 2.8471 | Test Acc 0.3455 | Time(s) 0.1254 | (Best 0.3455)
Epoch 00007 | Loss 2.7396 | Test Acc 0.3650 | Time(s) 0.1201 | (Best 0.3650)
Epoch 00008 | Loss 2.6345 | Test Acc 0.3809 | Time(s) 0.1160 | (Best 0.3809)
Epoch 00009 | Loss 2.5304 | Test Acc 0.4091 | Time(s) 0.1126 | (Best 0.4091)
Epoch 00010 | Loss 2.4265 | Test Acc 0.4787 | Time(s) 0.1099 | (Best 0.4787)
Epoch 00011 | Loss 2.3225 | Test Acc 0.5609 | Time(s) 0.1076 | (Best 0.5609)
Epoch 00012 | Loss 2.2185 | Test Acc 0.5943 | Time(s) 0.1057 | (Best 0.5943)

Epoch 00107 | Loss 0.3606 | Test Acc 0.9329 | Time(s) 0.0856 | (Best 0.9329)
Epoch 00108 | Loss 0.3593 | Test Acc 0.9331 | Time(s) 0.0856 | (Best 0.9331)
Epoch 00109 | Loss 0.3580 | Test Acc 0.9332 | Time(s) 0.0856 | (Best 0.9332)
Epoch 00110 | Loss 0.3568 | Test Acc 0.9333 | Time(s) 0.0856 | (Best 0.9333)
Epoch 00111 | Loss 0.3556 | Test Acc 0.9333 | Time(s) 0.0855 | (Best 0.9333)
Epoch 00112 | Loss 0.3544 | Test Acc 0.9334 | Time(s) 0.0855 | (Best 0.9334)
Epoch 00113 | Loss 0.3533 | Test Acc 0.9337 | Time(s) 0.0855 | (Best 0.9337)
Epoch 00114 | Loss 0.3521 | Test Acc 0.9338 | Time(s) 0.0855 | (Best 0.9338)
Epoch 00115 | Loss 0.3510 | Test Acc 0.9338 | Time(s) 0.0855 | (Best 0.9338)
Epoch 00116 | Loss 0.3499 | Test Acc 0.9341 | Time(s) 0.0854 | (Best 0.9341)
Epoch 00117 | Loss 0.3488 | Test Acc 0.9342 | Time(s) 0.0854 | (Best 0.9342)
Epoch 00118 | Loss 0.3477 | Test Acc 0.9344 | Time(s) 0.0854 | (Best 0.9344)
Epoch 00119 | Loss 0.3467 | Test Acc 0.9346 | Time(s) 0.0854 | (Best 0.9346)

Epoch 00214 | Loss 0.2897 | Test Acc 0.9397 | Time(s) 0.0843 | (Best 0.9397)
Epoch 00215 | Loss 0.2893 | Test Acc 0.9397 | Time(s) 0.0843 | (Best 0.9397)
Epoch 00216 | Loss 0.2889 | Test Acc 0.9397 | Time(s) 0.0842 | (Best 0.9397)
Epoch 00217 | Loss 0.2886 | Test Acc 0.9397 | Time(s) 0.0842 | (Best 0.9397)
Epoch 00218 | Loss 0.2882 | Test Acc 0.9398 | Time(s) 0.0842 | (Best 0.9398)
Epoch 00219 | Loss 0.2879 | Test Acc 0.9398 | Time(s) 0.0842 | (Best 0.9398)
Epoch 00220 | Loss 0.2875 | Test Acc 0.9399 | Time(s) 0.0842 | (Best 0.9399)
Epoch 00221 | Loss 0.2872 | Test Acc 0.9399 | Time(s) 0.0842 | (Best 0.9399)
Epoch 00222 | Loss 0.2868 | Test Acc 0.9399 | Time(s) 0.0842 | (Best 0.9399)
Epoch 00223 | Loss 0.2865 | Test Acc 0.9399 | Time(s) 0.0842 | (Best 0.9399)
Epoch 00224 | Loss 0.2862 | Test Acc 0.9400 | Time(s) 0.0842 | (Best 0.9400)
Epoch 00225 | Loss 0.2858 | Test Acc 0.9401 | Time(s) 0.0842 | (Best 0.9401)
Epoch 00226 | Loss 0.2855 | Test Acc 0.9401 | Time(s) 0.0842 | (Best 0.9401)

Epoch 00321 | Loss 0.2609 | Test Acc 0.9420 | Time(s) 0.0838 | (Best 0.9420)
Epoch 00322 | Loss 0.2607 | Test Acc 0.9421 | Time(s) 0.0838 | (Best 0.9421)
Epoch 00323 | Loss 0.2605 | Test Acc 0.9421 | Time(s) 0.0838 | (Best 0.9421)
Epoch 00324 | Loss 0.2603 | Test Acc 0.9421 | Time(s) 0.0838 | (Best 0.9421)
Epoch 00325 | Loss 0.2601 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00326 | Loss 0.2599 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00327 | Loss 0.2596 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00328 | Loss 0.2594 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00329 | Loss 0.2592 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00330 | Loss 0.2590 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00331 | Loss 0.2588 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00332 | Loss 0.2587 | Test Acc 0.9422 | Time(s) 0.0838 | (Best 0.9422)
Epoch 00333 | Loss 0.2585 | Test Acc 0.9423 | Time(s) 0.0838 | (Best 0.9423)

Epoch 00428 | Loss 0.2429 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00429 | Loss 0.2427 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00430 | Loss 0.2426 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00431 | Loss 0.2425 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00432 | Loss 0.2423 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00433 | Loss 0.2422 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00434 | Loss 0.2420 | Test Acc 0.9434 | Time(s) 0.0836 | (Best 0.9435)
Epoch 00435 | Loss 0.2419 | Test Acc 0.9435 | Time(s) 0.0835 | (Best 0.9435)
Epoch 00436 | Loss 0.2418 | Test Acc 0.9435 | Time(s) 0.0835 | (Best 0.9435)
Epoch 00437 | Loss 0.2416 | Test Acc 0.9435 | Time(s) 0.0835 | (Best 0.9435)
Epoch 00438 | Loss 0.2415 | Test Acc 0.9435 | Time(s) 0.0835 | (Best 0.9435)
Epoch 00439 | Loss 0.2414 | Test Acc 0.9435 | Time(s) 0.0835 | (Best 0.9435)
Epoch 00440 | Loss 0.2412 | Test Acc 0.9436 | Time(s) 0.0835 | (Best 0.9436)

...

Epoch 00642 | Loss 0.2203 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00643 | Loss 0.2202 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00644 | Loss 0.2201 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00645 | Loss 0.2200 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00646 | Loss 0.2199 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00647 | Loss 0.2199 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00648 | Loss 0.2198 | Test Acc 0.9451 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00649 | Loss 0.2197 | Test Acc 0.9451 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00650 | Loss 0.2196 | Test Acc 0.9452 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00651 | Loss 0.2195 | Test Acc 0.9451 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00652 | Loss 0.2195 | Test Acc 0.9451 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00653 | Loss 0.2194 | Test Acc 0.9451 | Time(s) 0.0833 | (Best 0.9452)
Epoch 00654 | Loss 0.2193 | Test Acc 0.9451 | Time(s) 0.0833 | (Best 0.9452)

Epoch 00749 | Loss 0.2124 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00750 | Loss 0.2124 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00751 | Loss 0.2123 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00752 | Loss 0.2123 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00753 | Loss 0.2122 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00754 | Loss 0.2121 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00755 | Loss 0.2121 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00756 | Loss 0.2120 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00757 | Loss 0.2119 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00758 | Loss 0.2119 | Test Acc 0.9456 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00759 | Loss 0.2118 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00760 | Loss 0.2117 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)
Epoch 00761 | Loss 0.2117 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9457)

Epoch 00856 | Loss 0.2062 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00857 | Loss 0.2062 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00858 | Loss 0.2061 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00859 | Loss 0.2061 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00860 | Loss 0.2060 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00861 | Loss 0.2060 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00862 | Loss 0.2059 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00863 | Loss 0.2059 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00864 | Loss 0.2058 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00865 | Loss 0.2058 | Test Acc 0.9459 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00866 | Loss 0.2057 | Test Acc 0.9459 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00867 | Loss 0.2057 | Test Acc 0.9459 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00868 | Loss 0.2056 | Test Acc 0.9459 | Time(s) 0.0832 | (Best 0.9459)

Epoch 00963 | Loss 0.2011 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00964 | Loss 0.2011 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00965 | Loss 0.2010 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00966 | Loss 0.2010 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00967 | Loss 0.2009 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00968 | Loss 0.2009 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00969 | Loss 0.2009 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00970 | Loss 0.2008 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00971 | Loss 0.2008 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00972 | Loss 0.2007 | Test Acc 0.9457 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00973 | Loss 0.2007 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00974 | Loss 0.2006 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)
Epoch 00975 | Loss 0.2006 | Test Acc 0.9458 | Time(s) 0.0832 | (Best 0.9459)

GCN(
  (layer1): GraphConv(in=602, out=16, normalization=both, activation=None)
  (layer2): GraphConv(in=16, out=41, normalization=both, activation=None)
)

## Save Hidden Features

In [25]:
l1_feats = activation['layer1'].cpu()
l2_feats = activation['layer2'].cpu()

save_weight('data/features/Reddit_feat_L1.txt',coo_matrix(l1_feats))
save_weight('data/features/Reddit_feat_L2.txt',coo_matrix(l2_feats))

activation.clear()
torch.save(model.state_dict(),'data/weight/reddit.pt')

### Save Layer Weight (State Dict) ###

#statedict = model.state_dict()
#L1 = statedict['layer1.weight'].to('cpu') + statedict['layer1.bias'].to('cpu')
#L2 = statedict['layer2.weight'].to('cpu') + statedict['layer2.bias'].to('cpu')

